In [1]:
import Agently
from datetime import datetime
from dotenv import load_dotenv
import os

load_dotenv()


[WARNING][2024-09-23 17:41:20.070] redis_rate_limiter.py:21 [t:130129511331648]: no redis installed, RedisRateLimiter unavailable


True

In [2]:

agent_factory = Agently.AgentFactory()


# 创建 Agent 实例
agent = (
    Agently.create_agent()
    .set_settings("current_model", "OAIClient")
    .set_settings("model.OAIClient.auth", { "api_key": f"{os.getenv('DEEPSEEK_API')}" })
    .set_settings("model.OAIClient.url", f"{os.getenv('DEEPSEEK_URL')}")
    .set_settings("model.OAIClient.options", { "model": "deepseek-chat" })
)
agent.alias_manager.print_alias_info()

{
    "register_tool": {
        "paramaters": [
            "tool_name",
            "desc",
            "args",
            "func",
            "categories",
            "require_proxy"
        ],
        "is_return_value": false,
        "docstring": "None",
        "agent_component": "Tool"
    },
    "call_tool": {
        "paramaters": [
            "tool_name",
            "args",
            "kwargs"
        ],
        "is_return_value": false,
        "docstring": "None",
        "agent_component": "Tool"
    },
    "set_tool_proxy": {
        "paramaters": [
            "proxy"
        ],
        "is_return_value": false,
        "docstring": "None",
        "agent_component": "Tool"
    },
    "add_public_tools": {
        "paramaters": [
            "tool_name_list"
        ],
        "is_return_value": false,
        "docstring": "None",
        "agent_component": "Tool"
    },
    "use_public_tools": {
        "paramaters": [
            "tool_name_list"
        ],
      

In [7]:
result = (
    agent
    .input("您能干什么")
    .start()
)

print(result)

作为一个通用领域的人工智能，我可以帮助您回答问题，提供信息，或者进行愉快的聊天。您不管有什么话题都可以找我聊聊~


In [8]:
# Create a meta data dict of those two tables:
## Usually we can get the meta data from our system by requesting APIs
## instead of typing them down below manually
meta_data = [
    {
        "table_name": "sales_purchase",
        "columns": [
            { "name": "company_name", "desc": "公司名称", "data type": "string", "example": "国贸能化" },
            { "name": "customer_name", "desc": "客户名称", "data type": "string", "example": "客户A" },
            { "name": "sales_amount", "desc": "销售金额", "data type": "float", "example": 32456.0 },
            { "name": "purchase_amount", "desc": "采购金额", "data type": "float", "example": 30000.0 },
            { "name": "gross_profit", "desc": "毛利润", "data type": "float", "example": 24567.0 },
            { "name": "gross_margin", "desc": "毛利率", "data type": "float", "example": 0.0756 },
            { "name": "sales_settlement_date", "desc": "销售结算日期", "data type": "date", "example": "2024-03-15" },
            { "name": "purchase_settlement_date", "desc": "采购结算日期", "data type": "date", "example": "2024-01-05" },
            { "name": "product_category", "desc": "商品类别", "data type": "string", "example": "煤炭" },
        ],
    }
]
## As you can see, this `meta_data` dict has a complex structure
## But it's OK, because Agently framework can help you handle the complex structure
## You can pass the dict to .info() or .input() like a variable directly
## instead of trying to figure out how to prompt or how to make model understand

# language = input("[Output Language]: ")
language="中文"
# agent = agent_factory.create_agent()
print("Ask questions about data using natural language.\nInput '#exit' if you want to stop.\n------\n")
while True:
    user_input = input("[Question]: ")
    if user_input == "#exit":
        break
    print("[Thinking]:")
    result = (
        agent
        .input(user_input)
        .info({ "database meta data": meta_data })
        .info({ "current date": datetime.now().date() })
        .instruct([
            "Generate SQL for MySQL database according {database meta data} to answer {input}",
            f"Language: { language }",
        ])
        .segment(
            "thinkings",
            ("String", "Your thinkings step by step about how to query data to answer {input}"),
            lambda data: print(data, end=""),
            is_streaming=True
        )
        .on_segment_done(
            "thinkings",
            lambda data: print("[SQL]:")
        )
        .segment(
            "SQL",
            ("String", "SQL String without explanation"),
            lambda data: print(data, end=""),
            is_streaming = True
        )
        .start()
    )
    print("\n------\n") 

Ask questions about data using natural language.
Input '#exit' if you want to stop.
------

[Thinking]:

为了回答“今年上半年集团各公司的销售额”，我们需要从数据库中提取销售数据。首先，我们需要筛选出今年上半年的销售记录。根据当前日期是2024年9月23日，今年上半年指的是2024年1月1日至2024年6月30日。然后，我们需要按公司名称分组，并计算每个公司的销售总额。
[SQL]:

SELECT company_name, SUM(sales_amount) AS total_sales
FROM sales_purchase
WHERE sales_settlement_date BETWEEN '2024-01-01' AND '2024-06-30'
GROUP BY company_name;
```
------

[Thinking]:

首先，我们需要从`sales_purchase`表中查询数据。根据输入，我们需要计算每个公司的毛利润总和，并按毛利润总和降序排列。为了实现这一点，我们可以使用`GROUP BY`子句按`company_name`分组，并使用`SUM`函数计算每个公司的毛利润总和。最后，使用`ORDER BY`子句按毛利润总和降序排列结果。
[SQL]:

SELECT company_name, SUM(gross_profit) AS total_gross_profit FROM sales_purchase GROUP BY company_name ORDER BY total_gross_profit DESC
------

[Thinking]:
